# Regressão Linear Múltipla: Trabalhando com Planilhas (2)
Nesta lição, vamos construir um modelo de regressão linear múltipla, porém utilizando uma outra base de dados - inclusive com dados faltantes! Assim, vamos enfatizar as etapas de pré-processamento dos dados.

## Importando dados
Vamos começar, como sempre, importando as bibliotecas e o conjunto de dados.

In [1]:
import numpy as np
import pandas as pd

dataset = pd.read_csv('./Data.csv')
dataset.head()

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,NaN,Yes


Neste exemplo, o dataset é formado por um conjunto de informações de compradores de um e-commerce fictício. Cada linha do CSV possui a informação de *Country* (País), *Age* (Idade) e *Salary* (Salário) de um consumidor. A última coluna (*Purchase*) indica se o consumidor comprou ou não na loja virtual. No total, o dataset é formado por 10 consumidores, ou seja, temos 10 observações.

Após a importação, precisamos separar o dataset em variável dependente (y) e variável independente (X).

In [2]:
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1].values

In [3]:
X

array([['France', 44.0, 72000.0],
       ['Spain', 27.0, 48000.0],
       ['Germany', 30.0, 54000.0],
       ['Spain', 38.0, 61000.0],
       ['Germany', 40.0, nan],
       ['France', 35.0, 58000.0],
       ['Spain', nan, 52000.0],
       ['France', 48.0, 79000.0],
       ['Germany', 50.0, 83000.0],
       ['France', 37.0, 67000.0]], dtype=object)

In [4]:
y

array(['No', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes'],
      dtype=object)

## Lidando com valores ausentes
Em aprendizado de máquina, é muito comum que o dataset tenha valores ausentes. Por esse motivo, é essencial adotarmos estratégias para resolução desse problema.

Existem duas formas principais de lidar com isso:

1. **Remoção:** consiste em remover as linhas com dados ausentes. em geral, essa estratégia é útil quando lidamos com grandes conjuntos de dados e poucos valores ausentes.
2. **Substituição:** consiste em substituir o valor ausente por outro valor. Nesse caso, pode-se usar valores como a média ou a mediana das demais observações, ou pelo valor que aparece com mais frequência na base.

No nosso exemplo, o `scikit-learn` possui um método, denominado `SimpleImputer`, que permite fazer a substituição. Vamos importar esse método e utilizá-lo.

In [5]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values = np.nan, strategy='mean')

Observe que o método possui alguns parâmetros. No caso, `missing_values` indica o "tipo" de valor que queremos substituir. Falando de maneira simplificada, estamos procurando um padrão na nossa base e esse padrão será substituído por algo. Nesse exemplo, estamos procurando por valores numéricos ausentes (`np.nan`, ou seja, *not a number*, NaN).

O outro argumento que usamos é `strategy`. No caso, indicamos que a substituição será feita pela média (`mean`). Poderíamos usar, alternativamente, a mediana (`median`), o valor mais frequente (`most_frequent`) ou, ainda, uma constante definida por nós mesmos (`constant`). Nesse último caso, teríamos que passar um parâmetro adicional, chamado `fill_value`, com esse valor definido por nós.

Uma vez invocado esse comando, devemos aplicá-lo à nossa base (no caso, aos valores de `X`). Para isso, vamos executar os seguintes comandos, indicando que essa busca e substituição deve considerar as três colunas de `X`.

In [6]:
imputer.fit(X[:,1:3])
X[:, 1:3] = imputer.transform(X[:,1:3])

Observe, agora, o resultado dessa transformação, que afetou um valor da coluna *Salary* e outro da coluna *Age*:

In [7]:
X

array([['France', 44.0, 72000.0],
       ['Spain', 27.0, 48000.0],
       ['Germany', 30.0, 54000.0],
       ['Spain', 38.0, 61000.0],
       ['Germany', 40.0, 63777.77777777778],
       ['France', 35.0, 58000.0],
       ['Spain', 38.77777777777778, 52000.0],
       ['France', 48.0, 79000.0],
       ['Germany', 50.0, 83000.0],
       ['France', 37.0, 67000.0]], dtype=object)

## Lidando com valores categóricos
Já vimos como fazer lidar com dados categórios. Vamos, novamente, usar o método `OneHotEncoder` para transformar a coluna `Country` de `X`.

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])],remainder = 'passthrough')
X = np.array(ct.fit_transform(X))

Note que passamos como parâmetro a coluna `[0]`, que se refere ao país. Como tínhamos três países. serão inseridas três colunas na base (e a coluna original será removida).

In [9]:
X

array([[1.0, 0.0, 0.0, 44.0, 72000.0],
       [0.0, 0.0, 1.0, 27.0, 48000.0],
       [0.0, 1.0, 0.0, 30.0, 54000.0],
       [0.0, 0.0, 1.0, 38.0, 61000.0],
       [0.0, 1.0, 0.0, 40.0, 63777.77777777778],
       [1.0, 0.0, 0.0, 35.0, 58000.0],
       [0.0, 0.0, 1.0, 38.77777777777778, 52000.0],
       [1.0, 0.0, 0.0, 48.0, 79000.0],
       [0.0, 1.0, 0.0, 50.0, 83000.0],
       [1.0, 0.0, 0.0, 37.0, 67000.0]], dtype=object)

Também precisamos transformar a variável dependente `y`, cujos valores são `Yes` e `No`. Nesse caso, já temos dados binários, ou seja, podemos simplesmente convertê-los em 0 e 1. Usamos, para isso, o método `LabelEncoder`.

In [10]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

Note que precisamos, primeiro, instanciar a classe `LabelEncoder` para, então, aplicar o método `fit_transform()`. Como resultado, obtemos:



In [11]:
y

array([0, 1, 0, 0, 1, 1, 0, 1, 0, 1])

## Separando os dados
Já podemos separar nossos dados em *treinamento* e *teste*, como vimos nas últimas lições. Novamente, utilizaremos a função `train_test_split` do `sklearn`, cujo resultado são quatro conjunto de dados: `X_train`, `X_test`, `y_train` e `y_test`.

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [13]:
X_train

array([[0.0, 1.0, 0.0, 40.0, 63777.77777777778],
       [1.0, 0.0, 0.0, 37.0, 67000.0],
       [0.0, 0.0, 1.0, 27.0, 48000.0],
       [0.0, 0.0, 1.0, 38.77777777777778, 52000.0],
       [1.0, 0.0, 0.0, 48.0, 79000.0],
       [0.0, 0.0, 1.0, 38.0, 61000.0],
       [1.0, 0.0, 0.0, 44.0, 72000.0],
       [1.0, 0.0, 0.0, 35.0, 58000.0]], dtype=object)

In [14]:
X_test

array([[0.0, 1.0, 0.0, 30.0, 54000.0],
       [0.0, 1.0, 0.0, 50.0, 83000.0]], dtype=object)

In [15]:
y_train

array([1, 1, 1, 0, 1, 0, 0, 1])

In [16]:
y_test

array([0, 0])

Da maneira como fizemos aqui, já poderíamos aplicar algum método de regressão sobre nossos dados. Observe, porém, que há algo "estranho" nessa base: há muita diferença na *grandeza* dos dados. Temos valores binários para representar os países e a saída desejada, valores com duas casas decimais para a idade e valores na casa dos milhares para o salário. Isso pode prejudicar a nossa previsão (**Exercício!**).

Vamos, então, aplicar mais uma etapa de pré-processamento sobre o nosso dataset.

## Escalonando os dados
Essa etapa (mais conhecida como *Feature Scaling*) consiste em ajustar as variáveis do dataset de forma a colocá-las em uma mesma escala.

Existem duas técnicas de *feature scaling* bastante conhecidas:

1. **Padronização:** tem o objetivo de colocar os dados em uma mesma escala, sendo que essa escala tem média 0 e desvio-padrão igual a 1;
2. **Normalização:** tem o objetivo de colocar os dados em uma mesma escala; no entanto os valores variam no intervalo de 0 a 1 (ou de -1 a 1, no caso de haver valores negativos nos dados). Essa técnica é mais apropriada para lidar com variáveis que já possuem o comportamento de uma distribuição normal.

No nosso exemplo, usamos a técnica de Padronização para as colunas *Age* e *Salary*.

In [17]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train[:, 3:] = sc.fit_transform(X_train[:,3:])
X_test[:, 3:] = sc.transform(X_test[:,3:])

In [18]:
print(X_train)

[[0.0 1.0 0.0 0.2630675731713538 0.1238147854838185]
 [1.0 0.0 0.0 -0.25350147960148617 0.4617563176278856]
 [0.0 0.0 1.0 -1.9753983221776195 -1.5309334063940294]
 [0.0 0.0 1.0 0.05261351463427101 -1.1114197802841526]
 [1.0 0.0 0.0 1.6405850472322605 1.7202971959575162]
 [0.0 0.0 1.0 -0.08131179534387283 -0.16751412153692966]
 [1.0 0.0 0.0 0.9518263102018072 0.9861483502652316]
 [1.0 0.0 0.0 -0.5978808481167128 -0.48214934111933727]]


In [19]:
print(X_test)

[[0.0 1.0 0.0 -1.4588292694047795 -0.9016629672292141]
 [0.0 1.0 0.0 1.984964415747487 2.139810822067393]]


## Exercícios

Com os dados já padronizados, agora
e a sua vez! Complete esse programa, fazendo as próximas etapas, ou seja: treine o modelo com os dados de treinamento e verifique a sua precisão com os dados de teste. Além disso, refaça esse exemplo *desconsiderando* a etapa de escalonamento dos dados e analise os resultados.

Em seguida, encontre outra base de dados (pode ser uma base do próprio `sklearn` ou alguma outra de seu interesse) e aplique as técnicas que você aprendeu até o momento. Repita todos os procedimentos de pré-processamente que forem necessários.

